In [7]:
import os
import torch
import torch.nn as nn 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU index

print("Available GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

Available GPUs: 1
GPU 0: Quadro P2000
cuda:0


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch 
import sys
sys.path.insert(1, os.path.join("..", "data"))
sys.path.insert(1, os.path.join("..", "utils"))
from data_utils import Dataset
from plot_utils import plot_image
from torch.utils.data import DataLoader

In [10]:
# Path to the data folder (update the variable to your path).
path_data=os.path.join("..", "data")
# Seed value
seed=1001

In [11]:
dataset=Dataset(path_data=path_data, seed=seed)
dataset.read_data()
dataset.get_statistics()

Parsing class: Cloud: 143it [00:10, 13.07it/s]
Parsing class: Edge: 97it [00:05, 16.17it/s]
Parsing class: Good: 64it [00:05, 12.61it/s]


,train,valid,test
cloud,100,24,19
edge,64,15,18
good,48,7,9


In [13]:
batch_size=32
# Train loader
train_loader = DataLoader(dataset.get_split("train"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Cross validation data loader
valid_loader = DataLoader(dataset.get_split("valid"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Test data loader
test_loader = DataLoader(dataset.get_split("test"), batch_size=batch_size, pin_memory=False, shuffle=True)

In [14]:
classes = ('cloud', 'edge', 'good')

Start of training loop

In [17]:
import torch.nn.functional as F 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def resize_tensor_images(images, size=(256, 256)):
    # Resize the batch of images
    return F.interpolate(images, size=size, mode='bilinear', align_corners=False)

def compute_mean_std(loader):
    # Computation of mean and standard deviation of batches
    mean = 0.
    std = 0.
    total_images_count = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count

    return mean, std

def normalize_images(images, mean, std):
    # Normalizing images with previously computed mean and standard deviation
    normalized_images = (images - mean.view(-1, 1, 1)) / std.view(-1, 1, 1)
    return normalized_images
    
def tensor_to_numpy(tensor):
    # Rescale the tensor to 0-1 range
    tensor = tensor - tensor.min()
    tensor = tensor / tensor.max()
    # Move the tensor to CPU if it's on GPU
    tensor = tensor.cpu()
    # Convert to numpy and transpose from CxHxW to HxWxC for visualization
    numpy_image = tensor.numpy()
    numpy_image = np.transpose(numpy_image, (1, 2, 0))

    return numpy_image

In [ ]:
batches_VAL = []
batches_TRL = []
batches_TST = []

UNPRO_batches_VAL = []
UNPRO_batches_TRL = []
UNPRO_batches_TST = []

In [18]:

def normalization(data_loader):
    for batch in data_loader:
        images, labels = batch
        resized_images = resize_tensor_images(images)
        UNPRO_batches.append((resized_images, labels))
        normalized_alldata_images = normalize_images(resized_images, mean, std)

        # Append the normalized images and their corresponding labels to the list
        batches.append((normalized_alldata_images, labels))

mean, std = compute_mean_std(test_loader)
for batch in valid_loader:
    images_VAL, labels_VAL = batch
    resized_images_VAL = resize_tensor_images(images_VAL)
    UNPRO_batches_VAL.append((resized_images_VAL, labels_VAL))
    normalized_alldata_images_VAL = normalize_images(resized_images_VAL, mean, std)

    # Append the normalized images and their corresponding labels to the list
    batches_VAL.append((normalized_alldata_images_VAL, labels_VAL))


for batch in train_loader:
    images_TRL, labels_TRL = batch
    resized_images_TRL = resize_tensor_images(images_TRL)
    UNPRO_batches_TRL.append((resized_images_TRL, labels_TRL))
    normalized_alldata_images_TRL = normalize_images(resized_images_TRL, mean, std)

    # Append the normalized images and their corresponding labels to the list
    batches_TRL.append((normalized_alldata_images_TRL, labels_TRL))
    

for batch in test_loader:
    images_TST, labels_TST = batch
    resized_images_TST = resize_tensor_images(images_TST)
    UNPRO_batches_TST.append((resized_images_TST, labels_TST))
    normalized_alldata_images_TST = normalize_images(resized_images_TST, mean, std)

    # Append the normalized images and their corresponding labels to the list
    batches_TST.append((normalized_alldata_images_TST, labels_TST))

NameError: name 'UNPRO_batches_VAL' is not defined